# EfficientNetB0 Model 

In [1]:
# Core Dependencies (for all models)
import os
import numpy as np
import matplotlib.pyplot as plt

# Deep Learning Dependencies
import tensorflow as tf
from sklearn.utils.class_weight import compute_class_weight
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import Model

# Custom Model Dependencies (for EfficientNet)
from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense, Dropout, BatchNormalization

### Load and Pre-process FER2013 Dataset

In [2]:
# Define the Paths 
dataset_train = 'FER2013/train'
dataset_test = 'FER2013/test'

In [3]:
# Define parameters
img_size = (48, 48)
batch_size = 64
input_shape = (img_size[0], img_size[1], 3)  # 3-channel input for EfficientNetB0

## Trial #4: EfficientNetB0 Model 

### Create Data Generators 

In [4]:
# Define Data Generators
train_datagen = ImageDataGenerator(
    rescale=1./255,
    validation_split=0.15,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.15,
    zoom_range=0.2,
    horizontal_flip=True
)

test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    dataset_train,
    target_size=img_size,
    color_mode='rgb',
    batch_size=batch_size,
    class_mode='categorical',
    subset='training',
    shuffle=True
)

val_generator = train_datagen.flow_from_directory(
    dataset_train,
    target_size=img_size,
    color_mode='rgb',
    batch_size=batch_size,
    class_mode='categorical',
    subset='validation',
    shuffle=True
)

test_generator = test_datagen.flow_from_directory(
    dataset_test,
    target_size=img_size,
    color_mode='rgb',
    batch_size=batch_size,
    class_mode='categorical',
    shuffle=False
)

Found 24406 images belonging to 7 classes.
Found 4303 images belonging to 7 classes.
Found 7178 images belonging to 7 classes.


### Load EfficientNetB0 Base

In [5]:
# Compute class weights
num_classes = len(train_generator.class_indices)
class_weights = compute_class_weight(
    class_weight='balanced',
    classes=np.unique(train_generator.classes),
    y=train_generator.classes
)
class_weights_dict = dict(enumerate(class_weights))

### Define the EfficientNetB0 Model

In [6]:
# Load EfficientNetB0 model (w/ ImageNet weights)
base_model = EfficientNetB0(weights='imagenet', include_top=False, input_shape=input_shape)

In [7]:
# Freeze all layers (initially)
for layer in base_model.layers:
    layer.trainable = False

# Add custom classifier head
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = BatchNormalization()(x)
x = Dense(512, activation='relu')(x)
x = Dropout(0.5)(x)
x = Dense(256, activation='relu')(x)
x = Dropout(0.3)(x)
output = tf.keras.layers.Dense(num_classes, activation='softmax')(x)

In [8]:
# Create the model
model_4 = Model(inputs=base_model.input, outputs=output)

# Define loss function with label smoothing
loss = tf.keras.losses.CategoricalCrossentropy(label_smoothing=0.05) 

# Compile the model (Test 4)
model_4.compile(optimizer=Adam(learning_rate=1e-3),
              loss=loss,
              metrics=['accuracy'])
model_4.summary()

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer         │ (None, 48, 48, 3) │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ rescaling           │ (None, 48, 48, 3) │          0 │ input_layer[0][0] │
│ (Rescaling)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ normalization       │ (None, 48, 48, 3) │          7 │ rescaling[0][0]   │
│ (Normalization)     │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ rescaling_1         │ (None, 48, 48, 3) │          0 │ normalization[0]… │
│ (Rescaling)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ stem_conv_pad       │ (None, 49, 49, 3) │          0 │ rescaling_1[0][0] │
│ (ZeroPadding2D)     │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ stem_conv (Conv2D)  │ (None, 24, 24,    │        864 │ stem_conv_pad[0]… │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ stem_bn             │ (None, 24, 24,    │        128 │ stem_conv[0][0]   │
│ (BatchNormalizatio… │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ stem_activation     │ (None, 24, 24,    │          0 │ stem_bn[0][0]     │
│ (Activation)        │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_dwconv      │ (None, 24, 24,    │        288 │ stem_activation[… │
│ (DepthwiseConv2D)   │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_bn          │ (None, 24, 24,    │        128 │ block1a_dwconv[0… │
│ (BatchNormalizatio… │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_activation  │ (None, 24, 24,    │          0 │ block1a_bn[0][0]  │
│ (Activation)        │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_se_squeeze  │ (None, 32)        │          0 │ block1a_activati… │
│ (GlobalAveragePool… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_se_reshape  │ (None, 1, 1, 32)  │          0 │ block1a_se_squee… │
│ (Reshape)           │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_se_reduce   │ (None, 1, 1, 8)   │        264 │ block1a_se_resha… │
│ (Conv2D)            │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_se_expand   │ (None, 1, 1, 32)  │        288 │ block1a_se_reduc… │
│ (Conv2D)            │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_se_excite   │ (None, 24, 24,    │          0 │ block1a_activati… │
│ (Multiply)          │ 32)               │            │ block1a_se_expan… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_project_co… │ (None, 24, 24,    │        512 │ block1a_se_excit

 Total params: 4,843,690 (18.48 MB)

 Trainable params: 791,559 (3.02 MB)

 Non-trainable params: 4,052,131 (15.46 MB)

### Train the Model

#### Phase 1: Train Frozen Base

In [9]:
# Initial training (top layer only)
initial_callbacks = [
    EarlyStopping(monitor='val_loss', patience=6, restore_best_weights=True),
    ReduceLROnPlateau(monitor='val_loss', patience=3, factor=0.5)
]

# Train the model
history = model_4.fit(
    train_generator,
    validation_data=val_generator,
    epochs=30,
    callbacks=initial_callbacks,
    class_weight=class_weights_dict
)

Epoch 1/30


/opt/anaconda3/envs/env1/lib/python3.11/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


382/382 ━━━━━━━━━━━━━━━━━━━━ 22s 51ms/step - accuracy: 0.1357 - loss: 2.2433 - val_accuracy: 0.0151 - val_loss: 1.9538 - learning_rate: 0.0010
Epoch 2/30
382/382 ━━━━━━━━━━━━━━━━━━━━ 20s 52ms/step - accuracy: 0.1056 - loss: 1.9626 - val_accuracy: 0.0151 - val_loss: 1.9563 - learning_rate: 0.0010
Epoch 3/30
382/382 ━━━━━━━━━━━━━━━━━━━━ 20s 53ms/step - accuracy: 0.0654 - loss: 1.9472 - val_accuracy: 0.0153 - val_loss: 1.9542 - learning_rate: 0.0010
Epoch 4/30
382/382 ━━━━━━━━━━━━━━━━━━━━ 23s 60ms/step - accuracy: 0.0537 - loss: 1.9601 - val_accuracy: 0.0151 - val_loss: 1.9514 - learning_rate: 0.0010
Epoch 5/30
382/382 ━━━━━━━━━━━━━━━━━━━━ 24s 62ms/step - accuracy: 0.0540 - loss: 1.9428 - val_accuracy: 0.0151 - val_loss: 1.9512 - learning_rate: 0.0010
Epoch 6/30
382/382 ━━━━━━━━━━━━━━━━━━━━ 23s 61ms/step - accuracy: 0.0320 - loss: 1.9541 - val_accuracy: 0.0151 - val_loss: 1.9510 - learning_rate: 0.0010
Epoch 7/30
382/382 ━━━━━━━━━━━━━━━━━━━━ 23s 60ms/step - accuracy: 0.0280 - loss: 1.9358

#### Phase 2: Fine-Tune Top Layers

In [10]:
# Fine-tune the model (unfreeze top EfficientNetB0 layers)
for layer in base_model.layers[-50:]:
    layer.trainable = True

# Compile for fine-tuning (lowered learning rate)
model_4.compile(optimizer=Adam(learning_rate=1e-5),
              loss=loss,
              metrics=['accuracy'])

# fine_tune_callbacks = [
#     tf.keras.callbacks.EarlyStopping(patience=8, restore_best_weights=True),
#     tf.keras.callbacks.ReduceLROnPlateau(patience=3, factor=0.5)
# ]

# Continue training (fine-tune entire model)
history_finetune = model_4.fit(
    train_generator,
    validation_data=val_generator,
    epochs=30,
    callbacks=initial_callbacks,
    class_weight=class_weights_dict
)

Epoch 1/30
382/382 ━━━━━━━━━━━━━━━━━━━━ 33s 74ms/step - accuracy: 0.0209 - loss: 1.9308 - val_accuracy: 0.0151 - val_loss: 1.9484 - learning_rate: 1.0000e-05
Epoch 2/30
382/382 ━━━━━━━━━━━━━━━━━━━━ 29s 77ms/step - accuracy: 0.0240 - loss: 1.9666 - val_accuracy: 0.0151 - val_loss: 1.9484 - learning_rate: 1.0000e-05
Epoch 3/30
382/382 ━━━━━━━━━━━━━━━━━━━━ 28s 74ms/step - accuracy: 0.0303 - loss: 1.9499 - val_accuracy: 0.0151 - val_loss: 1.9484 - learning_rate: 1.0000e-05
Epoch 4/30
382/382 ━━━━━━━━━━━━━━━━━━━━ 29s 77ms/step - accuracy: 0.0343 - loss: 1.9440 - val_accuracy: 0.0151 - val_loss: 1.9484 - learning_rate: 1.0000e-05
Epoch 5/30
382/382 ━━━━━━━━━━━━━━━━━━━━ 30s 78ms/step - accuracy: 0.0336 - loss: 1.9569 - val_accuracy: 0.0151 - val_loss: 1.9484 - learning_rate: 5.0000e-06
Epoch 6/30
382/382 ━━━━━━━━━━━━━━━━━━━━ 32s 83ms/step - accuracy: 0.0311 - loss: 1.9504 - val_accuracy: 0.0151 - val_loss: 1.9484 - learning_rate: 5.0000e-06
Epoch 7/30
382/382 ━━━━━━━━━━━━━━━━━━━━ 32s 84ms/ste

### Evaluate the Model

In [11]:
# Final evaluation (model #4)
test_loss, test_accuracy = model_4.evaluate(test_generator)
print(f"Trial Test #4 Accuracy: {test_accuracy:.4f}")

  1/113 ━━━━━━━━━━━━━━━━━━━━ 6s 54ms/step - accuracy: 0.0000e+00 - loss: 1.9461

/opt/anaconda3/envs/env1/lib/python3.11/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


113/113 ━━━━━━━━━━━━━━━━━━━━ 4s 38ms/step - accuracy: 0.0301 - loss: 1.9472
Trial Test #4 Accuracy: 0.0155


### Plot the Model on Training Histroy 

In [ ]:
# Plot Accuracy 
plt.figure(figsize=(12, 4))
plt.subplot(1, 2, 1)
plt.plot(history.history['accuracy'], label='Train Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.title('Model Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()
plt.show()

In [ ]:
# Plot Loss
plt.subplot(1, 2, 2)
plt.plot(history.history['loss'], label='Train Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.title('Model Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.tight_layout()
plt.show()

**Question: Was this model successful?** 

**Answer:**
* Accuracy:  
* Loss: 